In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

# Femellistes

## Scrapping les postes de blogs

In [18]:
# Fonction pour récupérer les informations d'une page donnée
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('div', class_='blog-basic-grid--text')
        for article in articles:
            title = article.find('h1', class_='blog-title').text.strip()
            author = article.find('span', class_='blog-author').text.strip()
            date = article.find('time', class_='blog-date').text.strip()
            categories = [cat.text.strip() for cat in article.find_all('a', class_='blog-categories')]
            excerpt = article.find('div', class_='blog-excerpt').text.strip()
            read_more_link = article.find('a', class_='blog-more-link')['href']

            article_url = f"https://www.femelliste.com{read_more_link}"
            article_response = requests.get(article_url)
            if article_response.status_code == 200:
                article_soup = BeautifulSoup(article_response.text, 'html.parser')
                main_content = article_soup.find('div', {'data-content-field': 'main-content'})
                full_text = main_content.get_text(strip=True)
            else:
                full_text = "Échec de la requête pour l'article."

            writer.writerow({
                'Titre': title,
                'Auteur': author,
                'Date': date,
                'Catégories': ', '.join(categories),
                'Extrait': excerpt,
                'Texte Complet': full_text,
                'URL Complète': article_url
            })
    else:
        print(f"Échec de la requête avec le code d'état: {response.status_code}")

# Ouvrir un fichier TSV en mode écriture avec un délimiteur tab
with open('./text/femelliste_articles.tsv', 'w', newline='', encoding='utf-8') as tsvfile:
    fieldnames = ['Titre', 'Auteur', 'Date', 'Catégories', 'Extrait', 'Texte Complet', 'URL Complète']
    writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()

    base_url = "https://www.femelliste.com/articles-femellisme-feminisme"

    while True:
        current_url = f"{base_url}"
        scrape_page(current_url)

        # Vérifier la présence du lien "Billets plus anciens"
        older_button = BeautifulSoup(requests.get(current_url).text, 'html.parser').find('div', class_='older')
        
        # Vérifier si older_button existe et si un lien 'rel=next' est présent
        next_page_link = older_button.find('a', rel='next')['href'] if older_button and older_button.find('a', rel='next') else None

        # Sortir de la boucle s'il n'y a pas de lien suivant ou si l'URL ne change pas
        if not next_page_link or next_page_link == base_url:
            break

        # Mettre à jour l'URL pour la prochaine itération
        base_url = f"https://www.femelliste.com{next_page_link}"


# TRADFEM

In [21]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# Fonction pour récupérer les informations d'une page donnée
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('div', class_='entry-thumbnail')

        for article in articles:
            article_url = article.find('a')['href']
            image_url = article.find('img')['src']

            article_response = requests.get(article_url)
            if article_response.status_code == 200:
                article_soup = BeautifulSoup(article_response.text, 'html.parser')

                title_element = article_soup.find('h1', class_='entry-title')
                title = title_element.text.strip() if title_element else "Titre non disponible"

                # Ajout pour récupérer la date
                date_element = article_soup.find('a', class_='entry-date')
                date = date_element.text.strip() if date_element else "Date non disponible"

                # Ajout pour récupérer l'auteur
                author_element = article_soup.find('a', class_='author')
                author = author_element.text.strip() if author_element else "Auteur non disponible"

                main_content = article_soup.find('div', {'id': 'content', 'class': 'site-content', 'role': 'main'})
                full_text = main_content.get_text(strip=True)
            else:
                full_text = "Échec de la requête pour l'article."

            writer.writerow({
                'Titre': title,
                'Date': date,
                'Auteur': author,
                'Image URL': image_url,
                'Texte Complet': full_text,
                'URL Complète': article_url
            })

            # Incrémenter le compteur
            global blog_count
            blog_count += 1

    else:
        print(f"Échec de la requête avec le code d'état: {response.status_code}")

# Ouvrir un fichier TSV en mode écriture avec un délimiteur tab dans le répertoire "./text/"
output_file_path = './text/tradfem_articles.tsv'
with open(output_file_path, 'w', newline='', encoding='utf-8') as tsvfile:
    fieldnames = ['Titre', 'Date', 'Auteur', 'Image URL', 'Texte Complet', 'URL Complète']
    writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()

    base_url = "https://tradfem.wordpress.com/"
    blog_count = 0

    while True:
        scrape_page(base_url)

        # Vérifier la présence du lien "Billets plus anciens"
        older_button = BeautifulSoup(requests.get(base_url).text, 'html.parser').find('div', class_='nav-previous')
        next_page_link = older_button.find('a')['href'] if older_button and older_button.find('a') else None

        # Sortir de la boucle s'il n'y a pas de lien suivant
        if not next_page_link:
            break

        # Mettre à jour l'URL pour la prochaine itération
        base_url = next_page_link

# Print le nombre de blogs scannés
print(f"Nombre de blogs scannés : {blog_count}")


KeyboardInterrupt: 

# Egalité et réconciliation
echec car la plus part des liens ne sont pas des postes avec du texte mais renvoie vers autres choses.

In [1]:
import requests
from bs4 import BeautifulSoup

# Base URL of the website
base_url = 'https://www.egaliteetreconciliation.fr/'

# Function to scrape a single page
def scrape_page(url):
    # Use requests to fetch the content of the website
    response = requests.get(url)
    response.raise_for_status()  # This will raise an exception for HTTP error codes

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <a> tags within <h4> tags
    post_links = soup.find_all('h4')
    links = [base_url + a.find('a')['href'] for a in post_links if a.find('a')]

    return links

# Initialize an empty list to hold all links
all_links = []

# Loop through all pages
for page in range(1, 88):  # 87 pages in total
    if page == 1:
        # The URL for the first page
        page_url = base_url + 'spip.php?page=alaune'
    else:
        # Calculate the debut_artg value for the current page
        debut_artg = (page - 1) * 10
        page_url = base_url + f'spip.php?page=alaune&debut_artg={debut_artg}#pagination_artg'
    
    # Scrape the current page
    links = scrape_page(page_url)
    all_links.extend(links)
    print(f"Scraped page {page} with {len(links)} links.")

# Optionally, print all collected links (commented out to avoid flooding the output)
# for link in all_links:
#     print(link)

print(f"Total links collected: {len(all_links)}")


Scraped page 1 with 10 links.
Scraped page 2 with 10 links.
Scraped page 3 with 10 links.
Scraped page 4 with 10 links.
Scraped page 5 with 10 links.
Scraped page 6 with 10 links.
Scraped page 7 with 10 links.
Scraped page 8 with 10 links.
Scraped page 9 with 10 links.
Scraped page 10 with 10 links.
Scraped page 11 with 10 links.
Scraped page 12 with 10 links.
Scraped page 13 with 10 links.
Scraped page 14 with 10 links.
Scraped page 15 with 10 links.
Scraped page 16 with 10 links.
Scraped page 17 with 10 links.
Scraped page 18 with 10 links.
Scraped page 19 with 10 links.
Scraped page 20 with 10 links.
Scraped page 21 with 10 links.
Scraped page 22 with 10 links.
Scraped page 23 with 10 links.
Scraped page 24 with 10 links.
Scraped page 25 with 10 links.
Scraped page 26 with 10 links.
Scraped page 27 with 10 links.
Scraped page 28 with 10 links.
Scraped page 29 with 10 links.
Scraped page 30 with 10 links.
Scraped page 31 with 10 links.
Scraped page 32 with 10 links.
Scraped page 33 w

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialize an empty list to hold dictionaries of the scraped data
scraped_data = []

# Loop through each URL in the all_links list
for url in all_links:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the title
        title_h1 = soup.find('h1', class_='crayon article-titre')
        title = title_h1.text.strip() if title_h1 else 'Title not found'
        
        # Extract the subtitle (as author in this context)
        subtitle_h2 = soup.find('h2', class_='crayon article-soustitre')
        author = subtitle_h2.text.strip() if subtitle_h2 else 'Author not found'
        
        # Extract the publication date
        pub_date = soup.find('abbr', class_='published')
        publication_date = pub_date['title'] if pub_date else 'Date not found'
        
        # Extract the main text
        main_text_div = soup.find('div', class_='crayon article-texte')
        main_text = main_text_div.text.strip() if main_text_div else 'Text not found'
        
        # Append the extracted information as a dictionary to the list
        scraped_data.append({
            'Title': title,
            'Author': author,
            'Publication Date': publication_date,
            'URL': url,
            'Text': main_text
        })

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(scraped_data)

# Display the first few rows of the DataFrame to verify
print(df.head())

# Optionally, save the DataFrame to a CSV file
df.to_csv('../data/text/EetR_articles.tsv', sep='\t', index=True)

             Title            Author      Publication Date  \
0  Title not found  Author not found  2024-01-23T09:48:56Z   
1  Title not found  Author not found  2024-01-15T09:09:34Z   
2  Title not found  Author not found  2024-01-12T19:18:54Z   
3  Title not found  Author not found  2023-12-29T13:32:20Z   
4  Title not found  Author not found  2023-12-26T22:55:00Z   

                                                 URL            Text  
0  https://www.egaliteetreconciliation.fr/C-est-p...  Text not found  
1  https://www.egaliteetreconciliation.fr/Soral-e...  Text not found  
2  https://www.egaliteetreconciliation.fr/Pierre-...  Text not found  
3  https://www.egaliteetreconciliation.fr/Faits-D...  Text not found  
4  https://www.egaliteetreconciliation.fr/Jean-Mi...  Text not found  


# La petite sirène

In [ ]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.observatoirepetitesirene.org"
articles_list_url = "https://www.observatoirepetitesirene.org/nospapiers"

# Send a GET request
response = requests.get(articles_list_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Assuming articles are linked within <a> tags directly within a specific class or id
# You'll need to inspect the HTML structure of the page to adjust the selector accordingly
article_links = []
for link in soup.select("selector-for-article-links"):  # Update this selector based on actual HTML
    href = link.get('href')
    if href and not href.startswith('http'):
        href = base_url + href  # Construct full URL if needed
    article_links.append(href)

print(article_links)

data = []  # Initialize an empty list to store article data

for url in article_links:
    response = requests.get(url)
    article_soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract title - Adjust the selector based on the actual HTML structure
    title = article_soup.find('tag-for-title', class_='class-for-title').text.strip()
    
    # Extract publication date - Adjust the selector based on the actual HTML structure
    pub_date = article_soup.find('tag-for-date', class_='class-for-date')['attribute-for-date']
    
    # Adjust author extraction based on the actual HTML structure or set statically if constant
    author = 'Default Author'  # Static example, adjust as needed
    
    # Extract the text - Adjust the selector based on the actual HTML structure
    text_parts = article_soup.select('selector-for-article-text p')
    text = ' '.join(part.text for part in text_parts)
    
    # Append the information as a dictionary to the data list
    data.append({
        'Title': title,
        'Publication Date': pub_date,
        'Author': author,
        'URL': url,
        'Text': text
    })

# Example to demonstrate data structure, remove or adjust as needed for actual use
for article in data[:1]:  # Print first article's data as an example
    print(article)


# Christine Le Doaré

In [3]:
import requests
from bs4 import BeautifulSoup

# URL du sommaire des articles
url = "https://christineld75.wordpress.com/sommaire-des-articles/"

# Envoyer une requête GET à l'URL
response = requests.get(url)

# Analyser le contenu de la page avec BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Trouver la div contenant les liens des articles
entry_content = soup.find('div', class_='entry-content')

# Trouver tous les éléments 'ul' dans cette div
lists = entry_content.find_all('ul')

# Liste pour stocker les URLs des articles
urls = []

# Parcourir chaque 'ul' pour extraire les liens
for ul in lists:
    links = ul.find_all('a')  # Trouver tous les liens dans chaque 'ul'
    for link in links:
        url = link.get('href')  # Extraire l'URL
        if url:  # S'assurer que l'URL est non vide
            urls.append(url)
links_to_remove = ["mailto:?subject=%5BArticle%20partag%C3%A9%5D%20Sommaire%20des%20articles&body=https%3A%2F%2Fchristineld75.wordpress.com%2Fsommaire-des-articles%2F&share=email",
"https://christineld75.wordpress.com/sommaire-des-articles/?share=facebook",
"https://christineld75.wordpress.com/sommaire-des-articles/?share=twitter",
"https://christineld75.wordpress.com/sommaire-des-articles/#print"
]

for link in links_to_remove:
    # Vérifier si le lien est dans la liste et le supprimer
    if link in urls:
        urls.remove(link)
# Affichage des liens récupérés
for link in urls:
    print(link)


https://christineld75.wordpress.com/2016/12/06/quel-le-presidentiable-aura-le-courage-de-le-dire/
https://christineld75.wordpress.com/2016/10/07/combien-y-a-t-il-de-feminismes/
https://christineld75.wordpress.com/2016/08/17/burkini-reanimez-les-feministes/
https://christineld75.wordpress.com/2016/08/04/politiques-intellectuels-medias-reprenez-vous-vite/
https://christineld75.wordpress.com/2016/06/19/orlando-tuerie-homophobe-ou-terrorisme-islamiste/
https://christineld75.wordpress.com/2016/04/06/une-date-historique-de-lemancipation-des-femmes-6-avril-2016-la-prostitution-enfin-abolie/
https://christineld75.wordpress.com/2016/03/08/8-mars-une-eprouvante-journee/
https://christineld75.wordpress.com/2016/01/13/le-viol-politique-de-femmes-occidentales-une-sale-guerre-de-civilisation/
https://christineld75.wordpress.com/2015/11/18/assez/
https://christineld75.wordpress.com/2015/11/03/vous-avez-bien-dit-dignite/
https://christineld75.wordpress.com/2015/09/12/feministes-ne-pas-denoncer-cest-ca

In [4]:
data = []  # Initialize an empty list to store article data

for url in urls:
    response = requests.get(url)
    article_soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract title with error handling
    title_element = article_soup.find('h3', class_='entry-title')
    title = title_element.text.strip() if title_element else 'Title Not Found'
    
    # Extract publication date with error handling
    pub_date_element = article_soup.find('abbr', class_='published')
    pub_date = pub_date_element['title'] if pub_date_element else 'Date Not Found'
    
    # Author is always 'Christine Le Doaré'
    author = 'Christine Le Doaré'
    
    # Extract the text
    text_parts = article_soup.select('div.entry-content p')
    text = ' '.join(part.text for part in text_parts) if text_parts else 'Text Not Found'
    
    # Append the information as a dictionary to the data list
    data.append({
        'Title': title,
        'Publication Date': pub_date,
        'Author': author,
        'URL': url,
        'Text': text
    })


# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a TSV file
df.to_csv('../data/text/christineld_articles.tsv', sep='\t', index=True)

# Observatoire de la petite sirène

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.observatoirepetitesirene.org"
articles_list_url = "https://www.observatoirepetitesirene.org/nospapiers"

# Send a request to the summary page
response = requests.get(articles_list_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Continuation du code
article_links = []

# Supposons que les liens soient enveloppés dans un élément `a` près du titre de l'article.
# Vous devrez adapter le sélecteur CSS ci-dessous selon la structure réelle de la page.
for block in soup.select(".FbwBsX"):  # Utilisez la classe du bloc pour localiser les titres
    # Recherche de l'élément `a` le plus proche ou à l'intérieur du bloc sélectionné
    # Ceci est un exemple générique; vous devrez peut-être ajuster le sélecteur
    link = block.find_parent('a')
    if link:
        article_url = link.get('href')
        # Assurez-vous que l'URL est complète; sinon, concaténez avec base_url
        if article_url and not article_url.startswith('http'):
            article_url = base_url + article_url
        article_links.append(article_url)

# Affichez ou traitez les liens des articles récupérés
for url in article_links:
    print(url)

https://www.observatoirepetitesirene.org/post/papier-de-jacqueline-schaeffer-n-4
https://www.observatoirepetitesirene.org/post/papier-de-christian-flavigny-n-3
https://www.observatoirepetitesirene.org/post/papier-de-françois-rastier-n-2
https://www.observatoirepetitesirene.org/post/papier-de-claudio-rubiliani-n-1


In [ ]:
from datetime import datetime
# Initialize a list to store scraped data
articles_data = []

# Function to scrape an individual article
def scrape_article(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # You would adjust the selectors below based on the actual HTML structure of your target pages
    title = soup.find('h1').text.strip()  # Example: Adjust selector as needed
    # Extract author
    author_block = soup.find('span', class_='tQ0Q1A user-name dlINDG')
    author = author_block.text if author_block else 'Unknown'
    
    # Extract publication date
    date_block = soup.find('span', class_='post-metadata__date time-ago')
    if date_block:
        # Parse the date assuming the format is like "30 janv."
        pub_date_str = date_block.text # + " 2024"  # Assuming year is 2024
        # Convert French month name to number and format the date
        pub_date = pub_date_str # datetime.strptime(pub_date_str, '%d %m %Y').strftime('%d/%m/%Y')
    else:
        pub_date = 'Unknown'


    # Initialize a variable to store the concatenated text
    article_text = ""
    
    # Iterate over blocks to extract text (this part needs customization based on the actual HTML structure)
    for block in soup.select('div[data-breakout="normal"]'):
        # Extract text or handle specific types of content here
        text_content = block.get_text(strip=True)
        article_text += text_content + " "    
    return {
        'title': title,
        'url':article_url,
        'publication_date': pub_date,
        'author': author,
        'text': article_text,
    }

# Iterate over article links and scrape each article
for url in article_links:
    article_data = scrape_article(url)
    articles_data.append(article_data)

# Convert the list of dictionaries to a DataFrame for easier manipulation and saving
df = pd.DataFrame(articles_data)

# Optionally, save the DataFrame to a CSV file
df.to_csv('../data/text/petitesirene_articles.tsv', sep='\t', index=True)


# Résistance Lesbienne

les publications commence en juillet 2021 et finissent en octobre 2023. Manque juin 2023, décembre 2022, mai 2022, avril 2022
il y des défauts dans le contenu scrappé je crois.

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

base_url = "https://resistancelesbienne.fr/"
start_date = datetime(2021, 7, 1)
end_date = datetime(2023, 10, 1)
skip_months = ["2023-06", "2022-12", "2022-05", "2022-04"]

# Générer les URLs des archives mensuelles
archive_urls = []
current_date = start_date
while current_date <= end_date:
    month_str = current_date.strftime("%Y/%m")
    if month_str not in skip_months:
        archive_urls.append(f"{base_url}{month_str}/")
    current_date += timedelta(days=32)
    current_date = current_date.replace(day=1)

# Fonction pour récupérer les liens des articles d'une archive
def get_article_links(archive_url):
    response = requests.get(archive_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article_links = [a['href'] for a in soup.find_all('a', class_='more-link')]
    return article_links

# Parcourir chaque archive et récupérer les liens
all_article_links = []
for url in archive_urls:
    article_links = get_article_links(url)
    all_article_links.extend(article_links)


# Lien à supprimer car en anglais
links_to_remove = ["https://resistancelesbienne.fr/2021/07/02/our-statement-about-lesbophobic-assault-at-paris-pride-26th-june-2021/",
                  "https://resistancelesbienne.fr/2021/07/16/traduction-en-allemand-bericht-vom-csd-in-paris-am-26-6-2021/"]


for link in links_to_remove:
    # Vérifier si le lien est dans la liste et le supprimer
    if link in all_article_links:
        all_article_links.remove(link)
# Affichage des liens récupérés
for link in all_article_links:
    print(link)


https://resistancelesbienne.fr/2021/07/27/protegeons-nos-filles-1/
https://resistancelesbienne.fr/2021/07/02/communique-agression-lesbophobe-a-la-marche-des-fiertes-paris-2021/
https://resistancelesbienne.fr/2021/07/02/notre-tract-marche-des-fiertes-paris-2021/
https://resistancelesbienne.fr/2021/08/27/pourquoi-je-suis-une-femme-lesbienne-et-pas-non-binaire-queer/
https://resistancelesbienne.fr/2021/09/14/communique-agression-lesbophobe-a-la-marche-des-fiertes-bordeaux-2021/
https://resistancelesbienne.fr/2021/09/02/des-bigotes-qui-devraient-revoir-leurs-traumas-ma-reponse-en-tant-que-survivante/
https://resistancelesbienne.fr/2021/10/27/une-avocate-lesbienne-de-43-ans-affirme-quelle-a-vu-des-hommes-malveillant-pretendant-etre-des-femmes-trans-intimider-des-jeunes-filles-lesbiennes-naives-et-vulnerables-agees-da-p/
https://resistancelesbienne.fr/2021/10/26/certaines-femmes-transgenres-nous-poussent-a-avoir-des-relations-sexuelles-bbc-news/
https://resistancelesbienne.fr/2021/10/11/la-f

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialisation d'une liste pour stocker les données
data = []

# Fonction pour scraper un article
def scrape_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Récupération du titre
    title = soup.find('h1', class_='entry-title').text.strip()
    
    # Récupération de la date de publication
    date = soup.find('time', class_='entry-date').get('datetime').split('T')[0]
    
    # Récupération et nettoyage du texte de l'article
    # Exclure les parties non désirées comme les articles similaires et les liens de partage
    content_section = soup.find('div', class_='entry-content')
    # Supprimer ou ignorer les sections indésirables si présentes
    for excluded_section in content_section.find_all(['div', 'footer'], class_=['jp-relatedposts', 'entry-footer', 'sharedaddy']):
        excluded_section.decompose()
    
    content = content_section.text.strip()
    
    # Ajout des données récupérées dans la liste
    data.append({
        'title': title,
        'date': date,
        'text': content,
        'author': 'résistance lesbienne'
    })

# Supposons que all_article_links est déjà défini avec les URLs des articles
for link in all_article_links:
    scrape_article(link)

# Création d'un DataFrame à partir de la liste des données
df = pd.DataFrame(data)

# Sauvegarde du DataFrame dans un fichier TSV
df.to_csv('../data/text/resistance_lesbienne_articles.tsv', sep='\t', index=False)
